Testing notebook to quickly prototype ideas and a keep a log of progress iA

Data from: http://www.sociopatterns.org/datasets/contacts-in-a-workplace/

Code from: https://github.com/iaciac/higher-order-NG

In [1]:
import numpy as np
from scipy import random
import networkx as nx
import pandas as pd
import random
import collections
import matplotlib.pyplot as plt
import os
import json
from time import time
%matplotlib inline

In [2]:
datasetlist = ['InVS']

In [3]:
def extract_networks(data_dir, dataset, n_minutes=5, original_nets=True):
    """Function that reads the edgelist (t, i, j) and returns
    a network aggregated at n_minutes snapshots as a dictionary of nx.Graph()s,
    having t as a key.
    If original_nets is set to True it also returns the original non-aggregated network."""
    
    #Reading the data and setting t0
    f = open(data_dir+'/tij_' + dataset +'.dat')
    (t0,i,j) = map(int,str.split(f.readline()))
    #Special temporal scale for these two Datasets
    if dataset not in ['LyonSchool','LH10']:
        t0 = t0*20
    f.close()
    
    #Aggregation on scale of x minutes
    delta_t = 20*3*n_minutes;   
    if original_nets==True:
        originalnetworks = {}
    aggnetworks = {}
    f = open(data_dir+'/tij_' + dataset +'.dat')
    for line in f:
        (t,i,j) = map(int,str.split(line))
        #Special temporal scale for these two Datasets
        if dataset not in ['LyonSchool','LH10']:
            t = t*20
        if original_nets==True:
            if t not in originalnetworks:
                originalnetworks[t] = nx.Graph()
            originalnetworks[t].add_edge(i,j)
        #this is a trick using the integer division in python
        aggtime = t0 + ((t-t0)/delta_t)*delta_t 
        if aggtime not in aggnetworks:
            aggnetworks[aggtime] = nx.Graph()
        aggnetworks[aggtime].add_edge(i,j)
    f.close();
    if original_nets==True:
        return originalnetworks, aggnetworks;
    else:
        return aggnetworks;

In [4]:
def extract_cliques(gs):
    listsaggcliques = {}
    #looping over the networks in temporal order
    for t in sorted(gs.keys()):
        listsaggcliques[t] = list(nx.find_cliques(gs[t]));
    #returning a dictionary with list of cliques as values
    return listsaggcliques;

In [5]:
def clique_weights(cliques):
    from collections import Counter;
    tot_c = [];
    for t in cliques:
        tot_c.extend(map(frozenset,cliques[t]))
    return Counter(tot_c);

In [6]:
def average_clique_size(ws):
    return np.sum(map(lambda x: 1.0 * ws[x] * len(x), ws.keys()))/np.sum(ws.values());

In [7]:
def clean_non_maximal(ws):
    sd = dict(zip(ws.keys(), map(len,ws.keys())));
    import operator
    sizes = set(map(len,ws.keys()));
    sorted_sd = sorted(sd.items(), key=operator.itemgetter(1));
    simplices = dict.fromkeys(list(sizes),[]);
    maximal_simplices = {};
    for x in ws:
        maximal = True;
        for xx in ws:
            if (len(x)<len(xx)):
                if (set(x)<set(xx)):
                    maximal=False;
                    break;
        if maximal:
            maximal_simplices[x] = ws[x];
    return maximal_simplices;

In [8]:
def save_cliques(ws, data_dir, dataset,n_minutes, thr=None):
    if thr==None:
        ls = map(list,ws.keys());
    else:
        ls = [list(x) for x in ws if ws[x]>=thr];
    jd = open(data_dir+'aggr_'+str(n_minutes)+'min_cliques_thr'+str(thr)+'_'+dataset+'.json','w')
    json.dump(ls,jd)
    jd.close()
    return;

In [9]:
dataset_dir = '/Users/ali/MSci Project/sociopatterns datasets/'
out_dir = '/Users/ali/MSci Project/social structure graph data/'

datasets = ['InVS']

n_minutes = 15
thrs = [1,3,5]

for dataset in datasets:
    for thr in thrs:
        aggs = extract_networks(dataset_dir, dataset, n_minutes, original_nets=False);
        cliques = extract_cliques(aggs)
        ws = clique_weights(cliques);
        maximal_cliques = clean_non_maximal(ws);
        save_cliques(maximal_cliques, out_dir, dataset, n_minutes, thr=thr)

In [10]:
print(len(aggs))
print(len(cliques))
print(len(ws))
print(len(maximal_cliques))

7104
7104
788
687


In [11]:
keys = maximal_cliques.keys()
weights = maximal_cliques.values()

weights = list(weights)
#weights = np.array(weights) / max(weights)

edges = []
keys = list(keys)
for i in range(len(keys)):
    key = list(keys[i])
    edges.append((key[0], key[1], weights[i]))

Used open in browser extension to run the html generated by the pyvis package in browser.

The graph is based on dataset of time spent face to face with other people, with a resolution of 20 seconds. This has been aaggregated into 15 minute chunks and any cliques found have been taken to be representitive of relationships and are then plotted with along with the strength of interaction.

In [12]:
from pyvis.network import Network

net2 = Network()
for node in edges:
    net2.add_node(node[0])
    net2.add_node(node[1])
    net2.add_edge(node[0], node[1],value = node[2])
net2.repulsion()
#net2.toggle_physics(False)
net2.show('high school.html')